# Data Processing
-----

## Objective
adlkfnaldf

----
#### External Libraries Import

In [1]:
import pandas as pd
import numpy as np
from functions import to_inches, impute
from os import listdir
from fancyimpute import IterativeImputer
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


#### Read Cleaned Dataset

In [2]:
df_pga = pd.read_csv('../Data/Sets/full_pga_data.csv')
df_pga.shape

(4648, 84)

In [3]:
# date to datetime
df_pga['date'] = pd.to_datetime(df_pga['date'])

In [4]:
# using a regex filter get a list of distance columns
distance_columns = df_pga.filter(
    regex = 'distance_from|approaches|proximity|longest_putts|approach_|average_'
).columns

##### This list of columns will be used in the future for converting features in the form:  ' 20'\ 6" '  to total inches as a float.

<br><br>
## <u> Handle Missing Data <u/>

### Method 1: Ignore Missing Data

In [5]:
# grab all columns with fewer missing values than 5% of the data
drop_rows = []
for col in df_pga.columns:
    if df_pga[col].isnull().sum() < 0.05 * df_pga.shape[0]:
        drop_rows.append(col)
        
# drop those rows
df_pga1 = df_pga.dropna(subset = drop_rows)

# create a dataframe with all columns without missing data
notnull = []
for col in df_pga1.columns:
    if df_pga1[col].isnull().sum() == 0:
        notnull.append(col)
df_notnull = df_pga1[notnull]
print(f'This method shrinks the dataset to {df_notnull.shape[0]} \
observations and {df_notnull.shape[1]} features.')

This method shrinks the dataset to 3616 observations and 60 features.


- Note that by doing this, the number of tournaments shrinks by 1,080 and the number of columns shrinks by 24. However, this smaller dataset contains no missing values.

#### Convert Datatypes

In [6]:
# convert distance columns to total inches using to_inches function in functions.py file
for col in df_notnull.columns:
    if col in distance_columns:
        df_notnull[col] = df_notnull[col].apply(lambda x: to_inches(x))
        
# finish to integer
df_notnull['finish'] = df_notnull['finish'].apply(lambda x: x.strip('T')).astype(int)

#### Save to csv for modeling

In [7]:
df_notnull.to_csv('../Data/Sets/model_one.csv', index = False)

<br><br>
### Method 2: Hot-Deck Imputation

In [8]:
# get a list of columns with more than 500 missing values and drop them
drop_cols = []
for col in df_pga.columns:
    if df_pga[col].isnull().sum() > 500:
        drop_cols.append(col)
df_pga2 = df_pga.drop(columns = drop_cols)
print(f'This method shrinks the number of features to {df_pga2.shape[1]} \
but keeps the {df_pga2.shape[0]} observations.')

This method shrinks the number of features to 69 but keeps the 4648 observations.


- This process gets rid of 15 columns that contain enough missing values to argue that those statistics will not influence golf performance. A large majority of these columns are approaches from the rough and approach shots from an unordinary distance.

#### Convert Datatypes

In [9]:
# convert distance columns to total inches
for col in df_pga2.columns:
    if col in distance_columns:
        df_pga2[col] = df_pga2[col].apply(lambda x: to_inches(x))
        
# finish to integer
df_pga2['finish'] = df_pga2['finish'].apply(lambda x: x.strip('T')).astype(int)

In [10]:
# get list of players again from directory
players = listdir('../Data/')

# remove unwanted files/folders
players.remove('.DS_Store') 
players.remove('.ipynb_checkpoints')
players.remove('Sets')
len(players)

50

#### Impute 

In [11]:
# loop through each column that contains missing values. Replace the missing 
# values with a random choice from each players distribution of that feature.
for col in df_pga2.columns:
    if df_pga2[col].isnull().sum() > 0:  
        new_vals = []
        for player in players:
            filled = impute(df_pga2, col, player)
            new_vals.extend(filled)
        df_pga2[col] = new_vals

- Hot-Deck imputation fills a missing value by choosing a random value from a the list of observed values for the corresponding player and feature. 

#### Save to csv for modeling

In [12]:
df_pga2.to_csv('../Data/Sets/model_two.csv', index = False)

<br><br>
### Method 3: Regression Imputation

#### Method 3 takes four steps:
- Step 1: Drop columns with too many missing values to add any significance.
- Step 2: Drop rows with missing values in columns that rarely have missing values.
- Step 3: Created binary encoded columns for columns that contain missing values:
    - 1 means the value is missing.
    - 0 means the value is not missing.
- Step 4: Use IterativeImputer to fill missing values with a prediction calculated by a regression using observed values for each tournament.

#### Step 1

In [13]:
# drop columns with overwhelming missing values
drop_cols = []
for col in df_pga.columns:
    if df_pga[col].isnull().sum() > 500:
        drop_cols.append(col)
df_pga3 = df_pga.drop(columns = drop_cols)
print('Drop the same 15 columns as in method 2.')

Drop the same 15 columns as in method 2.


#### Step 2

In [14]:
# grab all columns with fewer missing values than 5% of the data
drop_rows = []
for col in df_pga3.columns:
    if df_pga3[col].isnull().sum() < 25:
        drop_rows.append(col)
        
# drop rows with missing data in those columns
df_pga3.dropna(subset = drop_rows, inplace = True)
print('Drops 110 rows.')

Drops 110 rows.


- Dropping these 110 rows significantly reduces the number of columns with missing data. It makes the encoding step much easier.

#### Convert datatypes

In [15]:
# convert distance columns to total inches
for col in df_pga3.columns:
    if col in distance_columns:
        df_pga3[col] = df_pga3[col].apply(lambda x: to_inches(x))
        
# finish to integer
df_pga3['finish'] = df_pga3['finish'].apply(lambda x: x.strip('T')).astype(int)

#### Step 3

In [16]:
# create an encoded column containing a 1 
# if the value is missing and a 0 if it is not
encode_list = []
for col in df_pga3.columns:
    if 25 < df_pga3[col].isnull().sum() < 500:
        encode_list.append(col)
        
for col in encode_list:
    df_pga3[f'{col}_encoded'] = df_pga3[col].map(lambda x: 1 if np.isnan(x) else 0)
print('Add 22 news encoded columns for remaining columns that contain missing valus.')

Add 22 news encoded columns for remaining columns that contain missing valus.


- Creating these binary columns serves to add information about missing values to the model. If the value was missing, the coefficient on the encoded column will account for the missingness. If it is not missing, it does not affect the model.

#### Step 4

In [17]:
# filter for only numeric columns to prepare for imputation
num_cols = [col for col in df_pga3.columns if col not in ['date', 'player', 'event']]
df_pga3 = df_pga3[num_cols]

# Use IterativeImputer to fill missing values. 
# This library fills the missing values with a prediction calculated
# from a regression based on other observed values
myimputer = IterativeImputer()
df_pga3_transformed = myimputer.fit_transform(df_pga3)
df_filled = pd.DataFrame(df_pga3_transformed)  

# rename the columns
df_filled.columns = df_pga3.columns

# add the non-numeric columns
df_filled[['date', 'player', 'event']] = df_pga[['date', 'player', 'event']]

#### Save to csv for modeling

In [18]:
df_filled.to_csv('../Data/Sets/model_three.csv', index = False)